In [191]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import learn
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn import datasets, linear_model
from sklearn import cross_validation
import numpy as np
import pandas as pd
from operator import itemgetter

df = pd.read_csv("https://raw.githubusercontent.com/EnterpriseDeepLearning/housing-prices-wide-and-deep/master/train_validate.csv")
df = df.fillna(0)
#print(df.dtypes) #floats, ints, and objects

dataset = df.values
non_categorical_features = dataset[:, [1,2,3,4,5,7,8,11,12,16,18,24]]
for col in ['acres','tax_gross_amount','total_stories','assessment_value_town', 'garage_capacity', 'year_built', 'common_land_acres']:
    df[col] = df[col].astype('int64')    
#print(df.dtypes)
categorical_features = df.select_dtypes(include=[object])
#print(categorical_features.head(3))
print(categorical_features.shape)
le = preprocessing.LabelEncoder()
categorical_features_2 = categorical_features.apply(le.fit_transform)
print(categorical_features_2.head(2))
enc = OneHotEncoder()
enc.fit(categorical_features_2)
# 3. Transform
onehotlabels = enc.transform(categorical_features_2).toarray()
print(onehotlabels.shape)
print(onehotlabels)
# as you can see, you've the same number of rows 891
# but now you've so many more columns due to how we changed all the categorical data into numerical data
for col in ['address','city','garage_type','surveyed', 'seasonal', 'water_body_type', 'water_frontage_length',
    'short_sale', 'garage', 'flood_zone', 'easements', 'current_use', 'covenants', 'basement_access_type', 'basement']:
    df[col] = categorical_features_2[col]
dataset = df.values
x = dataset[:,0:27]
print(x)
print(x.shape)
y = dataset[:,27:28]
print(y.shape)
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(
x, y, test_size=0.2, random_state=42)

total_len = X_train.shape[0]

(206, 15)
   address  city  garage_type  surveyed  seasonal  water_body_type  \
0      159     2            4         2         0                0   
1      123     2            3         2         0                0   

   water_frontage_length  short_sale  garage  flood_zone  easements  \
0                      0           1       1           0          0   
1                      0           0       1           0          0   

   current_use  covenants  basement_access_type  basement  
0            1          1                     1         2  
1            0          0                     3         1  
(206, 259)
[[ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 ..., 
 [ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]]
[[  1   3   3 ...,   0   1   2]
 [  2   4   3 ...,   0   3   1]
 [  3   4   3 ...,   0   1   2]
 ..., 
 [204   5   3 ...,   0   0   0]
 [205   3   1 ...,   0   1   2]
 [206  

In [32]:
# Parameters
learning_rate = 0.001
training_epochs = 500
batch_size = 10
display_step = 1
dropout_rate = 0.9
# Network Parameters
n_hidden_1 = 32 # 1st layer number of features
n_hidden_2 = 200 # 2nd layer number of features
n_hidden_3 = 200
n_hidden_4 = 256
n_input = X_train.shape[1]
n_classes = 1

# tf Graph input
x = tf.placeholder("float", [None, 27])
y = tf.placeholder("float", [None])

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)

    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)

    # Hidden layer with RELU activation
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)

    # Hidden layer with RELU activation
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.relu(layer_4)

    # Output layer with linear activation
    out_layer = tf.matmul(layer_4, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], 0, 0.1)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], 0, 0.1)),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3], 0, 0.1)),
    'h4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4], 0, 0.1)),
    'out': tf.Variable(tf.random_normal([n_hidden_4, n_classes], 0, 0.1))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1], 0, 0.1)),
    'b2': tf.Variable(tf.random_normal([n_hidden_2], 0, 0.1)),
    'b3': tf.Variable(tf.random_normal([n_hidden_3], 0, 0.1)),
    'b4': tf.Variable(tf.random_normal([n_hidden_4], 0, 0.1)),
    'out': tf.Variable(tf.random_normal([n_classes], 0, 0.1))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)
tf.transpose(pred)

# Define loss and optimizer
cost = tf.reduce_mean(tf.square(pred-y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Launch the graph
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(total_len/batch_size)
        # Loop over all batches
        for i in range(total_batch-1):
            batch_x = X_train[i*batch_size:(i+1)*batch_size]
            batch_y = Y_train[i*batch_size:(i+1)*batch_size]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c, p = sess.run([optimizer, cost, pred], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch

        # sample prediction
        label_value = batch_y
        estimate = p
        err = label_value-estimate
        print ("num batch:", total_batch)

        # Display logs per epoch step
        if epoch % display_step == 0:
            print ("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
            print ("[*]----------------------------")
            for i in xrange(3):
                print ("label value:", label_value[i], \
                    "estimated value:", estimate[i])
            print ("[*]============================")

    print ("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = sess.run(cost, feed_dict={x:X_test, y: Y_test})
    print(accuracy)
    predicted_vals = sess.run(pred, feed_dict={x: X_test})

Instructions for updating:
Use `tf.global_variables_initializer` instead.


ValueError: could not convert string to float: Yes